# Spatio-temporal kriging klasa za imputaciju nedostajućih vrednosti polena


## Učitavanje biblioteka

In [3]:
# Standardne biblioteke 
from datetime import timedelta
import warnings
from itertools import combinations

# Obrada podataka
import numpy as np
import pandas as pd

# Machine learning modeli, metrika i evaluacija
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors

# Statističke i matematičke funkcije
from scipy.optimize import curve_fit
from scipy.spatial.distance import cdist
from scipy.special import inv_boxcox
from scipy.stats import boxcox

# Time series modeliranje
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

# Progres bar za petlje
from tqdm import tqdm

# Isključivanje upozorenja
warnings.filterwarnings('ignore')

## Definisanje Spatio-temporal kriging klasa za imputaciju nedostajućih vrednosti polena

In [ ]:
class SpatioTemporalKriging(BaseEstimator):
    """
    SpatioTemporalKriging klasa za imputaciju vrednosti polena koristeći:
    - log1p ili Box-Cox transformaciju,
    - trend + sezonske  + meteorološke exogene prediktore,
    - spatio-temporal kriging za modelovanje reziduala.
    """

    def __init__(self, transform='boxcox', exog_cols=None,  bin_delta_space=40, bin_delta_time=5, value_col="value", date_col="date"):
        assert transform in ['log', 'boxcox', None], "Transformacija mora biti 'log', 'boxcox' ili None."
        self.transform = transform
        self.exog_cols = exog_cols if exog_cols is not None else []  # meteo promenljive
        self.bin_delta_space = bin_delta_space  # širina bina za prostorni variogram
        self.bin_delta_time = bin_delta_time    # širina bina za vremenski variogram
        self.value_col = value_col # naziv kolone za vrednost
        self.date_col = date_col # naziv vremenske kolone
        
    @staticmethod
    def variogram_model(h, C0, C, a):
        """Eksponencijalni model variograma."""
        return C0 + C * (1 - np.exp(-h / a))

    @staticmethod
    def gamma_fitted(h, C0, C, a):
        """Proračun variograma za zadate parametre."""
        return C0 + C * (1 - np.exp(-h / a))

    @staticmethod
    def log_rmse(y_true, y_pred):
        return np.sqrt(np.mean((np.log1p(y_pred/30) - np.log1p(y_true/30)) ** 2))

    @staticmethod
    def estimate_k(g_s, g_t, g_emp):
        """Procena parametra k spatio-temp variograma."""
        numerator = g_s + g_t - g_emp
        denominator = g_s * g_t + 1e-8
        return np.clip(np.mean(numerator / denominator), 0.1, 1.5)
    
    @staticmethod    
    def _bin_and_average(distances, semivariances, delta):
        """Binovanje udaljenosti i prosečna semivarijansa po binu."""
        max_distance = distances.max()
        bins = np.arange(0, max_distance + delta, delta)
        df = pd.DataFrame({'distance': distances, 'semivariance': semivariances})
        df['bin'] = pd.cut(df['distance'], bins=bins, include_lowest=True)
        grouped = df.groupby('bin').agg({'distance': 'mean', 'semivariance': 'mean'}).dropna().reset_index()
        return grouped['distance'].values, grouped['semivariance'].values 
    
    def _apply_transform(self, series, loc=None):
        """
        Transformacija niza.
        """
        if self.transform == 'log':
            transformed = np.log1p(series/30)
            self.lambda_map_[loc] = None
        elif self.transform == 'boxcox':
            safe_val = series + 1e-1
            transformed, lmbda = boxcox(safe_val)
            if lmbda<0:
                transformed = np.log(safe_val)
                lmbda=0
            self.lambda_map_[loc] = lmbda
        else:  # None
            transformed = series
            self.lambda_map_[loc] = None
        return transformed

    def _inverse_transform(self, series, loc=None):
        """
        Inverzna transformacija niza.
        """
        if self.transform == 'log':
            return 30*np.expm1(series)
        elif self.transform == 'boxcox':
            lmbda = self.lambda_map_.get(loc, 1.0)
            return inv_boxcox(series, lmbda) - 1e-1
        return series
    
    def _check_exog_cols(self, df):
        """
        Provera da li df sadrži sve exog kolone.
        """
        missing_cols = [col for col in self.exog_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Nedostaju egzogene kolone u df: {missing_cols}")

    def fit(self, df, meteo_df=pd.DataFrame()):
        """
        Treniranje modela:
        - Fit linearne regresije na transofrmisanim vrednostima
        - Fit variograma za prostor i vreme
        """

        # Provera da li meteo_df sadrži sve exog kolone
        self._check_exog_cols(meteo_df)

        df = df.copy()
        self.min_date_ = pd.to_datetime(df[self.date_col].min().strftime("%Y-01-01"))
        df["t"] = (df[self.date_col] - self.min_date_).dt.days

        # Inicijalizacija mapa za Box-Cox lambda, regresione i determinističke procese
        self.lambda_map_ = {}
        self.models_ = {}
        self.dp_models_ = {}
        processed = []

        # Petlja kroz sve lokacije i treniranje lokalnih regresionih modela
        for loc, group in df.groupby("location"):
            if group[self.value_col].notna().sum() < 5:
                continue

            df_loc = group.copy()

            # Merge meteoroloških podataka za lokaciju
            if self.exog_cols:
                meteo_loc = meteo_df[meteo_df["location"] == loc].copy()
                df_loc = df_loc.merge(meteo_loc, on=[self.date_col, "location"], how="left")

            # Transformacija
            df_loc['transform'] = self._apply_transform(df_loc[self.value_col], loc=loc)

            # Furije + trend + exog regresija
            fourier = CalendarFourier(freq='YE', order=2)
            dp = DeterministicProcess(
                index=df_loc[self.date_col],
                constant=True,
                order=0,
                seasonal=False,
                additional_terms=[fourier],
                drop=True
            )
            X_fourier = dp.in_sample()

            X_full = X_fourier.reset_index().copy()
            X_full = pd.merge(X_full, df_loc.loc[:, [self.date_col, "t"] + self.exog_cols], on=self.date_col)
            X_full.drop(columns=self.date_col, inplace=True)

            # Fit linearne regresije na transformisanim vrednostima
            y = df_loc["transform"]
            model = LinearRegression().fit(X_full, y)

            # Predikcija i izračunavanje reziduala
            df_loc["fitted"] = model.predict(X_full)
            df_loc["residual"] = df_loc["transform"] - df_loc["fitted"]

            # Čuvanje modela i determinističkog procesa po lokaciji
            self.models_[loc] = model
            self.dp_models_[loc] = dp
            processed.append(df_loc)

        df = pd.concat(processed)
        self.train_df_ = df.copy()

        # Kriging fitting
        sample = df.sample(n=min(1000, len(df)), random_state=42)
        coords = sample[["latitude", "longitude"]].values * 111
        times = sample["t"].values
        values = sample["residual"].values
        years = sample[self.date_col].dt.year.values.reshape(-1, 1)

        # Priprema parova tačaka za variogram
        pairs = list(combinations(range(len(sample)), 2))
        spatial_distances, semivariances_space = [], []
        temporal_distances, semivariances_time = [], []

        for i, j in pairs:
            d = np.linalg.norm(coords[i] - coords[j])
            gamma_ij = 0.5 * (values[i] - values[j]) ** 2
            if (times[i] == times[j]) and (d < 300):
                spatial_distances.append(d)
                semivariances_space.append(gamma_ij)
            if (d < 1) and (years[i] == years[j]) and (abs(times[i] - times[j]) < 50):
                td = np.abs(times[i] - times[j])
                temporal_distances.append(td)
                semivariances_time.append(gamma_ij)

        # Binovanje za stabilniji fit variograma
        spatial_distances_binned, semivariances_space_binned = self._bin_and_average(
            np.array(spatial_distances), np.array(semivariances_space), self.bin_delta_space)
        temporal_distances_binned, semivariances_time_binned = self._bin_and_average(
            np.array(temporal_distances), np.array(semivariances_time), self.bin_delta_time)

        # Fit prostornih i vremenskih variograma
        self.params_space_, _ = curve_fit(self.variogram_model, spatial_distances_binned, semivariances_space_binned,
                                p0=[0.1, 1.0, 30], bounds=([0, 0, 0], [np.inf, np.inf, np.inf]))
        self.params_time_, _ = curve_fit(self.variogram_model, temporal_distances_binned, semivariances_time_binned,
                                p0=[0.1, 1.0, 10], bounds=([0, 0, 0], [np.inf, np.inf, np.inf]))
        
        # Procena parametra k za spatio-temporal variogram
        g_s_sample, g_t_sample, g_total_sample = [], [], []
        for i, j in pairs:
            if (years[i] == years[j]) and (abs(times[i] - times[j]) < 50) and (np.linalg.norm(coords[i] - coords[j]) < 300):
                d_s = np.linalg.norm(coords[i] - coords[j])
                d_t = np.abs(times[i] - times[j])
                g_s = self.gamma_fitted(np.array([d_s]), *self.params_space_)[0]
                g_t = self.gamma_fitted(np.array([d_t]), *self.params_time_)[0]
                g_emp = 0.5 * (values[i] - values[j]) ** 2
                g_s_sample.append(g_s)
                g_t_sample.append(g_t)
                g_total_sample.append(g_emp)

        self.k_ = self.estimate_k(np.array(g_s_sample), np.array(g_t_sample), np.array(g_total_sample))

        return self

    def predict(self, df_test, meteo_df=None):
        """
        Predikcija na novom skupu podataka.
        """
        # Provera da li meteo_df sadrži sve exog kolone
        self._check_exog_cols(meteo_df)
        
        df_test = df_test.copy()
        df_test["t"] = (df_test[self.date_col] - self.min_date_).dt.days

        # Merge meteoroloških podataka ako postoje
        if self.exog_cols:
            df_test = df_test.merge(meteo_df, on=[self.date_col, "location"], how="left")
        
        # Priprema poznatih i nepoznatih tačaka za kriging
        X_known = self.train_df_[["latitude", "longitude", "t"]].values
        z_known = self.train_df_["residual"].values
        X_unknown = df_test[["latitude", "longitude", "t"]].values

        # Skaliranje vremena za spatio-temporal distance
        a_s = self.params_space_[2]
        a_t = self.params_time_[2]
        time_scale = np.clip(a_s / a_t, 10, 50)

        X_known_scaled = np.hstack([X_known[:, :2] * 111, X_known[:, 2:3] * time_scale])
        X_unknown_scaled = np.hstack([X_unknown[:, :2] * 111, X_unknown[:, 2:3] * time_scale])

        # Pronalaženje najbližih suseda
        nn = NearestNeighbors(n_neighbors=50, metric='euclidean')
        nn.fit(X_known_scaled)
        distances, indices = nn.kneighbors(X_unknown_scaled)

        # Filtiranje suseda unutar max distance
        max_distance = 200
        indices = [idxs[dists < max_distance] if np.any(dists < max_distance) else [] for dists, idxs in zip(distances, indices)]

        residual_pred = []
        for i, idx in enumerate(indices):
            if len(idx) < 5:
                residual_pred.append(0.0)
                continue

            X_k = X_known[idx]
            z_k = z_known[idx]
            x_u = X_unknown[i:i + 1]

            # Izračunavanje spatio-temporal variograma za predikciju
            d_s = np.linalg.norm(X_k[:, :2] * 111 - x_u[:, :2] * 111, axis=1)
            d_t = np.abs(X_k[:, 2] - x_u[:, 2])
            g_s = self.gamma_fitted(d_s, *self.params_space_)
            g_t = self.gamma_fitted(d_t, *self.params_time_)
            g = g_s + g_t - self.k_ * g_s * g_t

            # Izračunavanje matrice kovarijansi
            D_s = cdist(X_k[:, :2] * 111, X_k[:, :2] * 111)
            D_t = cdist(X_k[:, 2:3], X_k[:, 2:3])
            G_s = self.gamma_fitted(D_s, *self.params_space_)
            G_t = self.gamma_fitted(D_t, *self.params_time_)
            G = G_s + G_t - self.k_ * G_s * G_t

            n_local = G.shape[0]
            K_local = np.zeros((n_local + 1, n_local + 1))
            K_local[:n_local, :n_local] = G
            K_local[n_local, :-1] = 1
            K_local[:-1, n_local] = 1
            K_local[n_local, n_local] = 0

            try:
                # Rešavanje sistema i predikcija reziduala
                g_vec = np.append(g, 1)
                weights = np.linalg.solve(K_local, g_vec)
                z_pred = np.dot(weights[:-1], z_k)
            except np.linalg.LinAlgError:
                z_pred = 0.0

            residual_pred.append(z_pred)

        residual_pred = np.array(residual_pred)
        
        
        # Ograničavanje reziduala po lokaciji
        iqr_stats = self.train_df_.groupby("location")["residual"].agg(
            q1=lambda x: np.percentile(x, 25),
            q3=lambda x: np.percentile(x, 75)
        )
        iqr_stats["iqr"] = iqr_stats["q3"] - iqr_stats["q1"]
        iqr_stats["upper"] = iqr_stats["q3"] + iqr_stats["iqr"]

        upper_bound_for_test = df_test["location"].map(iqr_stats["upper"]).fillna(np.inf).values
        residual_pred = np.minimum(residual_pred, upper_bound_for_test)

        # Rekonstrukcija finalnih predikcija
        return self.reconstruct(df_test, residual_pred)

    def reconstruct(self, df, residual_pred):
        """
        Rekonstrukcija originalnih vrednosti iz fitted + residual + Box-Cox inverse.
        """
        locs = df["location"].values
        fitted = np.zeros(len(df))

        # Generisanje fitted vrednosti po lokaciji korišćenjem determinističkih procesa i linearnih modela
        for loc in np.unique(locs):
            idx = np.where(locs == loc)[0]
            if loc in self.models_:
                dp = self.dp_models_[loc]
                X_season = dp.out_of_sample(steps=len(idx), forecast_index=df.iloc[idx][self.date_col])
                
                X_full = pd.concat([X_season.reset_index(drop=True),
                                    df.iloc[idx][["t"] + self.exog_cols].reset_index(drop=True)], axis=1).fillna(0)
                
                # Predikcija fitted vrednosti pomoću treniranog regresionog modela
                fitted[idx] = self.models_[loc].predict(X_full)
                
        # Kombinovanje fitted vrednosti i predikovanih reziduala
        value_pred = fitted + residual_pred
        val = np.zeros(len(df))

        # Inverzna transformacija u originalni domen po lokaciji
        for i, loc in enumerate(locs):
            val[i] = self._inverse_transform(value_pred[i], loc=loc)

        # Zaokruživanje i ograničavanje vrednosti
        val = np.round(val, 0)
        val = np.clip(val, 0, None)
        val = np.nan_to_num(val, nan=0)
        return val

    def evaluate(self, y_true, y_pred):
        return self.log_rmse(y_true, y_pred)

    def cross_validate(self, df, meteo_df = None, k=5):
        """
        K-fold cross-validation evaluacija modela.
        """
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        scores = []
        for train_idx, test_idx in tqdm(kf.split(df), desc="k-ti fold"):
            train_df = df.iloc[train_idx].copy()
            test_df = df.iloc[test_idx].copy()
            self.fit(train_df, meteo_df)

            # Predikcija na test skupu
            y_pred = self.predict(test_df, meteo_df)
            y_true = test_df[self.value_col].values
            
            # Izračunavanje log RMSE i čuvanje rezultata
            scores.append(self.log_rmse(y_true, y_pred))

        return np.mean(scores), np.std(scores)
    

## Definisanje SpatioTemporalKrigingStandard klase za imputaciju sa standardizacijom 

In [ ]:
class SpatioTemporalKrigingStandard(SpatioTemporalKriging):
    """
    SpatioTemporalKriging klasa za imputaciju vrednosti polena koristeći:
    - transformaciju na standardizovanim podacima,
    - trend + sezonske (Fourier) + meteorološke exogene prediktore,
    - spatio-temporal kriging za modelovanje reziduala.
    """
        
    def _apply_transform(self, series):
        """
        Transformacija niz.
        """
        if self.transform == 'log':
            transformed = np.log1p(series/30)
        elif self.transform == 'boxcox':
            safe_val =  series + 1e-1
            transformed, self.shared_lambda_ = boxcox(safe_val)
            if self.shared_lambda_ <0:
                transformed = np.log(safe_val)
                self.shared_lambda_ = 0
        else:
            transformed =  series
        return transformed

    def _inverse_transform(self, series):
        """
        Inverzna transformacija niza.
        """
        if self.transform == 'log':
            return 30*np.expm1(series)
        elif self.transform == 'boxcox':
            return inv_boxcox(series, self.shared_lambda_) - 1e-1
        return series

    def fit(self, df, meteo_df=None):
        """
        Treniranje modela:
        - Skalira vrednosti po lokaciji
        - Izračunava Box-Cox transformaciju sa zajedničkom lambdom
        - Fit linearne regresije (trend + sezonsko + exog)
        - Fit variograma za prostor i vreme
        """

        # Provera da li meteo_df sadrži sve exog kolone
        self._check_exog_cols(meteo_df)
        
        df = df.copy()
        self.min_date_ = pd.to_datetime(df[self.date_col].min().strftime("%Y-01-01"))
        df["t"] = (df[self.date_col] - self.min_date_).dt.days

        # Inicijalizacija mapa za Box-Cox lambda, regresione i determinističke procese
        self.models_ = {}
        self.dp_models_ = {}
        self.scale_map_ = {}
        all_scaled_values = []
        processed = []

        # Skaliranje vrednosti po lokaciji
        for loc, group in df.groupby("location"):
            if group[self.value_col].notna().sum() < 5:
                continue
            
            df_loc= group.copy()
            # Merge meteo podatke za lokaciju
            if self.exog_cols:
                meteo_loc = meteo_df[meteo_df["location"] == loc].copy()
                df_loc = df_loc.merge(meteo_loc, on=[self.date_col, "location"], how="left")
            
            # Standardizacija vrednosti po lokaciji
            scale = np.std(df_loc[self.value_col])
            df_loc["value_scaled"] = df_loc[self.value_col] / scale
            self.scale_map_[loc] = scale
            all_scaled_values.append(df_loc)
            
        df_scaled = pd.concat(all_scaled_values)
        df_transformed = df_scaled.copy()
        df_transformed['transform'] = self._apply_transform(df_scaled['value_scaled'])
        

        # Petlja kroz sve lokacije i treniranje lokalnih regresionih modela
        for loc, group in df_transformed.groupby("location"):
            if group["transform"].notna().sum() < 5:
                continue

            df_loc = group.copy()

            # Furije + trend + exog regresija
            fourier = CalendarFourier(freq='YE', order=2)
            dp = DeterministicProcess(
                index=df_loc[self.date_col],
                constant=True,
                order=0,
                seasonal=False,
                additional_terms=[fourier],
                drop=True
            )
            X_fourier = dp.in_sample()

            X_full = X_fourier.reset_index().copy()
            X_full = pd.merge(X_full, df_loc.loc[:, [self.date_col, "t"] + self.exog_cols], on=self.date_col)
            X_full.drop(columns=self.date_col, inplace=True)

            # Fit linearne regresije na transformisanim vrednostima
            y = df_loc["transform"]
            model = LinearRegression().fit(X_full, y)

            # Predikcija i izračunavanje reziduala
            df_loc["fitted"] = model.predict(X_full)
            df_loc["residual"] = df_loc["transform"] - df_loc["fitted"]

            # Čuvanje modela i determinističkog procesa po lokaciji
            self.models_[loc] = model
            self.dp_models_[loc] = dp
            processed.append(df_loc)

        df = pd.concat(processed)
        self.train_df_ = df.copy()

        # Kriging fitting
        sample = df.sample(n=min(1000, len(df)), random_state=42)
        coords = sample[["latitude", "longitude"]].values * 111
        times = sample["t"].values
        values = sample["residual"].values
        years = sample[self.date_col].dt.year.values.reshape(-1, 1)

        # Priprema parova tačaka za variogram
        pairs = list(combinations(range(len(sample)), 2))
        spatial_distances, semivariances_space = [], []
        temporal_distances, semivariances_time = [], []

        for i, j in pairs:
            d = np.linalg.norm(coords[i] - coords[j])
            gamma_ij = 0.5 * (values[i] - values[j]) ** 2
            if (times[i] == times[j]) and (d < 300):
                spatial_distances.append(d)
                semivariances_space.append(gamma_ij)
            if (d < 1) and (years[i] == years[j]) and (abs(times[i] - times[j]) < 50):
                td = np.abs(times[i] - times[j])
                temporal_distances.append(td)
                semivariances_time.append(gamma_ij)

        # Binovanje za stabilniji fit variograma
        spatial_distances_binned, semivariances_space_binned = self._bin_and_average(
            np.array(spatial_distances), np.array(semivariances_space), self.bin_delta_space)
        temporal_distances_binned, semivariances_time_binned = self._bin_and_average(
            np.array(temporal_distances), np.array(semivariances_time), self.bin_delta_time)

        # Fit prostornih i vremenskih variograma
        self.params_space_, _ = curve_fit(self.variogram_model, spatial_distances_binned, semivariances_space_binned,
                                p0=[0.1, 1.0, 30], bounds=([0, 0, 0], [np.inf, np.inf, np.inf]))
        self.params_time_, _ = curve_fit(self.variogram_model, temporal_distances_binned, semivariances_time_binned,
                                p0=[0.1, 1.0, 10], bounds=([0, 0, 0], [np.inf, np.inf, np.inf]))
        
        # Procena parametra k za spatio-temporal variogram
        g_s_sample, g_t_sample, g_total_sample = [], [], []
        for i, j in pairs:
            if (years[i] == years[j]) and (abs(times[i] - times[j]) < 50) and (np.linalg.norm(coords[i] - coords[j]) < 300):
                d_s = np.linalg.norm(coords[i] - coords[j])
                d_t = np.abs(times[i] - times[j])
                g_s = self.gamma_fitted(np.array([d_s]), *self.params_space_)[0]
                g_t = self.gamma_fitted(np.array([d_t]), *self.params_time_)[0]
                g_emp = 0.5 * (values[i] - values[j]) ** 2
                g_s_sample.append(g_s)
                g_t_sample.append(g_t)
                g_total_sample.append(g_emp)

        self.k_ = self.estimate_k(np.array(g_s_sample), np.array(g_t_sample), np.array(g_total_sample))

        return self

    def predict(self, df_test, meteo_df=None):
        """
        Predikcija na novom skupu podataka.
        """
        # Provera da li meteo_df sadrži sve exog kolone
        self._check_exog_cols(meteo_df)
        
        df_test = df_test.copy()
        df_test["t"] = (df_test[self.date_col] - self.min_date_).dt.days

        # Merge meteoroloških podataka ako postoje
        if self.exog_cols:
            df_test = df_test.merge(meteo_df, on=[self.date_col, "location"], how="left")
        
        # Priprema poznatih i nepoznatih tačaka za kriging
        X_known = self.train_df_[["latitude", "longitude", "t"]].values
        z_known = self.train_df_["residual"].values
        X_unknown = df_test[["latitude", "longitude", "t"]].values

        # Skaliranje vremena za spatio-temporal distance
        a_s = self.params_space_[2]
        a_t = self.params_time_[2]
        time_scale = np.clip(a_s / a_t, 10, 50)

        X_known_scaled = np.hstack([X_known[:, :2] * 111, X_known[:, 2:3] * time_scale])
        X_unknown_scaled = np.hstack([X_unknown[:, :2] * 111, X_unknown[:, 2:3] * time_scale])

        # Pronalaženje najbližih suseda
        nn = NearestNeighbors(n_neighbors=50, metric='euclidean')
        nn.fit(X_known_scaled)
        distances, indices = nn.kneighbors(X_unknown_scaled)

        # Filtiranje suseda unutar max distance
        max_distance = 200
        indices = [idxs[dists < max_distance] if np.any(dists < max_distance) else [] for dists, idxs in zip(distances, indices)]

        residual_pred = []
        for i, idx in enumerate(indices):
            if len(idx) < 5:
                residual_pred.append(0.0)
                continue

            X_k = X_known[idx]
            z_k = z_known[idx]
            x_u = X_unknown[i:i + 1]

            # Izračunavanje spatio-temporal variograma za predikciju
            d_s = np.linalg.norm(X_k[:, :2] * 111 - x_u[:, :2] * 111, axis=1)
            d_t = np.abs(X_k[:, 2] - x_u[:, 2])
            g_s = self.gamma_fitted(d_s, *self.params_space_)
            g_t = self.gamma_fitted(d_t, *self.params_time_)
            g = g_s + g_t - self.k_ * g_s * g_t

            # Izračunavanje matrice kovarijansi
            D_s = cdist(X_k[:, :2] * 111, X_k[:, :2] * 111)
            D_t = cdist(X_k[:, 2:3], X_k[:, 2:3])
            G_s = self.gamma_fitted(D_s, *self.params_space_)
            G_t = self.gamma_fitted(D_t, *self.params_time_)
            G = G_s + G_t - self.k_ * G_s * G_t

            n_local = G.shape[0]
            K_local = np.zeros((n_local + 1, n_local + 1))
            K_local[:n_local, :n_local] = G
            K_local[n_local, :-1] = 1
            K_local[:-1, n_local] = 1
            K_local[n_local, n_local] = 0

            try:
                # Rešavanje sistema i predikcija reziduala
                g_vec = np.append(g, 1)
                weights = np.linalg.solve(K_local, g_vec)
                z_pred = np.dot(weights[:-1], z_k)
            except np.linalg.LinAlgError:
                z_pred = 0.0

            residual_pred.append(z_pred)

        residual_pred = np.array(residual_pred)
        
        
        # Ograničavanje reziduala po lokaciji
        iqr_stats = self.train_df_.groupby("location")["residual"].agg(
            q1=lambda x: np.percentile(x, 25),
            q3=lambda x: np.percentile(x, 75)
        )
        iqr_stats["iqr"] = iqr_stats["q3"] - iqr_stats["q1"]
        iqr_stats["upper"] = iqr_stats["q3"] + 2 * iqr_stats["iqr"]

        upper_bound_for_test = df_test["location"].map(iqr_stats["upper"]).fillna(np.inf).values
        residual_pred = np.minimum(residual_pred, upper_bound_for_test)

        # Rekonstrukcija finalnih predikcija
        return self.reconstruct(df_test, residual_pred)

    def reconstruct(self, df, residual_pred):
        """
        Rekonstrukcija originalnih vrednosti iz fitted + residual + Box-Cox inverse.
        """
        locs = df["location"].values
        fitted = np.zeros(len(df))

        # Generisanje fitted vrednosti po lokaciji korišćenjem determinističkih procesa i linearnih modela
        for loc in np.unique(locs):
            idx = np.where(locs == loc)[0]
            if loc in self.models_:
                dp = self.dp_models_[loc]
                X_season = dp.out_of_sample(steps=len(idx), forecast_index=df.iloc[idx][self.date_col])
                
                X_full = pd.concat([X_season.reset_index(drop=True),
                                    df.iloc[idx][["t"] + self.exog_cols].reset_index(drop=True)], axis=1).fillna(0)
                
                # Predikcija fitted vrednosti pomoću treniranog regresionog modela
                fitted[idx] = self.models_[loc].predict(X_full)

        # Kombinovanje fitted vrednosti i predikovanih reziduala
        value_pred = fitted + residual_pred
        val = self._inverse_transform(value_pred)

        # Reskaliranje po lokaciji
        scales = np.array([self.scale_map_.get(loc, 1) for loc in locs])
        val = val * scales

        # Zaokruživanje i ograničavanje vrednosti
        val = np.round(val, 0)
        val = np.clip(val, 0, None)
        val = np.nan_to_num(val, nan=0)
        return val

## Definisanje IDW klase za imputaciju vrednosti pomoću Inverse Distance Weighting metode

In [6]:
class IDW(BaseEstimator):
    """
    IDW (Inverse Distance Weighting) klasa za imputaciju vrednosti polena koristeći:
    - Pronalazi prostorne susede unutar istog dana i računa težinski prosek
      obrnuto proporcionalan udaljenosti (d^(-power)).
    - Ako nema dovoljno suseda tog dana uzima najbliži datum iste lokacije.
    """
    def __init__(self, min_neighbors=5, power=1):
        self.min_neighbors = min_neighbors # Minimalan broj suseda potreban za IDW
        self.power = power # stepen udaljenosti u IDW formuli

    @staticmethod
    def log_rmse(y_true, y_pred):
        return np.sqrt(np.mean((np.log1p(y_pred/30) - np.log1p(y_true/30)) ** 2))

    def fit(self, df):
        self.train_df_ = df.copy()
        self.train_by_date_ = self.train_df_.groupby("date")
        self.train_by_location_ = self.train_df_.groupby("location")
        return self

    def predict(self, test_df):
        """
        Predikcija vrednosti na test skupu pomoću IDW metode ili fallback strategije
        """
        preds = []
        test_df = test_df.copy()

        # Grupisanje test tačaka po datumu radi bržeg pronalaženja suseda
        for test_date, group in test_df.groupby("date"):
            # Dohvatanje podataka za isti datum ako postoje
            same_day = self.train_by_date_.get_group(test_date) if test_date in self.train_by_date_.groups else pd.DataFrame()
            test_coords = group[["latitude", "longitude"]].values

            if len(same_day) >= self.min_neighbors:
                # Ako ima dovoljno suseda istog dana, koristi IDW
                train_coords = same_day[["latitude", "longitude"]].values
                dists = cdist(test_coords, train_coords)
                weights = 1 / (dists ** self.power + 1e-8)
                weighted_vals = weights @ same_day["value"].values
                preds_batch = weighted_vals / weights.sum(axis=1)
                preds.extend(preds_batch)
            else:
                ## Fallback: koristi najbliži datum iste lokacije
                for _, row in group.iterrows():
                    if row["location"] in self.train_by_location_.groups:
                        loc_group = self.train_by_location_.get_group(row["location"])
                        loc_group = loc_group.copy()
                        loc_group["dt_diff"] = np.abs((loc_group["date"] - row["date"]).dt.days)
                        closest = loc_group.loc[loc_group["dt_diff"].idxmin()]
                        preds.append(closest["value"])
                    else:
                        preds.append(np.nan)

        return np.array(preds)

    def cross_validate(self, df, k=5):
        """
        K-fold cross-validation evaluacija modela.
        """
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        scores = []
        df = df.dropna(subset=["value"])
        
        for train_idx, test_idx in tqdm(kf.split(df), desc="k-ti fold"):
            train_df = df.iloc[train_idx].copy()
            test_df = df.iloc[test_idx].copy()
            self.fit(train_df)

            # Predikcija na test skupu
            y_pred = self.predict(test_df)
            y_true = test_df["value"].values

            # Izračunavanje log RMSE i čuvanje rezultata
            scores.append(self.log_rmse(y_true, y_pred))

        return np.mean(scores), np.std(scores)


## Definisanje NaiveTemporalNearest klase za imputaciju na osnovu najbližeg vremenskog suseda po lokaciji

In [ ]:
class NaiveTemporalNearest(BaseEstimator):
    """
    NaiveTemporalNearest klasa za imputaciju vrednosti polena koristeći najbliži vremenski sused.
    """
    
    @staticmethod
    def log_rmse(y_true, y_pred):
        return np.sqrt(np.mean((np.log1p(y_pred/30) - np.log1p(y_true/30)) ** 2))
    
    def fit(self, df):
        self.train_df_ = df.copy()
        self.train_by_location_ = self.train_df_.groupby("location")
        return self

    def predict(self, test_df):
        """
        Predikcija vrednosti na test skupu korišćenjem najbližeg vremenskog suseda.
        """
        test_df = test_df.copy()
        test_df["value"] = np.nan  # mesto za predikcije

        # Iteracija kroz svaku lokaciju u test skupu
        for loc, group in test_df.groupby("location"):
            if loc in self.train_by_location_.groups:
                train_loc = self.train_by_location_.get_group(loc).copy()
                train_loc = train_loc[["date", "value"]].dropna()

                test_dates = pd.to_datetime(group["date"]).values
                train_dates = pd.to_datetime(train_loc["date"]).values

                # Kreiranje matrice vremenskih razlika (test x train)
                diff = np.abs(test_dates[:, None] - train_dates[None, :])
                idx_min = np.argmin(diff, axis=1)

                # Mapiranje najbližih vrednosti na test tačke
                values = train_loc["value"].values[idx_min]
                test_df.loc[group.index, "value"] = values
            else:
                test_df.loc[group.index, "value"] = np.nan

        return test_df["value"].values

    def cross_validate(self, df, k=5):
        """
        K-fold cross-validation evaluacija modela.
        """
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        scores = []
        df = df.dropna(subset=["value"])
        
        for train_idx, test_idx in tqdm(kf.split(df), desc="k-ti fold"):
            train_df = df.iloc[train_idx].copy()
            test_df = df.iloc[test_idx].copy()
            self.fit(train_df)

            # Predikcija na test skupu
            y_pred = self.predict(test_df)
            y_true = test_df["value"].values

            # Izračunavanje log RMSE i čuvanje rezultata
            scores.append(self.log_rmse(y_true, y_pred))

        return np.mean(scores), np.std(scores)


## Učitavanje podataka

In [8]:
data = pd.read_csv('pollen_data_with_imputed_dates.csv', parse_dates=['date'])
locations = pd.read_csv('location.csv')
data = data.merge(locations, on='location', how ='left')
train_df = data[data.imputed==0]
test_df = data[data.imputed==1]
meteo_df = pd.read_csv('meteo\meteo_df.csv', parse_dates=['date'])

## Evaluacija modela

In [16]:
# Lista svih alergena
unique_allergens = train_df["allergen"].dropna().unique()

df_results = pd.DataFrame(index=unique_allergens)

# Kolone koje ćemo popuniti
columns = [
    "SpatioTemporalKriging_mean_bc", "SpatioTemporalKriging_std_bc",
    "SpatioTemporalKriging_mean_log", "SpatioTemporalKriging_std_log",
    "SpatioTemporalKrigingStand_mean_bc", "SpatioTemporalKrigingStand_std_bc",
    "SpatioTemporalKrigingStand_mean_log", "SpatioTemporalKrigingStand_std_log",
    "SpatioTemporalKrigingStand_mean_exog_bc", "SpatioTemporalKrigingStand_std_exog_bc",
    "SpatioTemporalKrigingStand_mean_exog_log", "SpatioTemporalKrigingStand_std_exog_log",
    "NaiveTemporalNearest_mean", "NaiveTemporalNearest_std",
    "IDWTemporalFallback_mean", "IDWTemporalFallback_std"
]

# Postavi prazne kolone
df_results = df_results.reindex(columns=columns)

# Petlja kroz alergene
for allergen in tqdm(unique_allergens, desc="Obrada alergena"): 
    df_allergen = train_df[train_df["allergen"] == allergen].copy()
    
    # SpatioTemporalKriging(boxcox)
    model_stk = SpatioTemporalKriging(transform='boxcox')
    mean_stk, std_stk = model_stk.cross_validate(df_allergen, meteo_df=meteo_df)
    df_results.loc[allergen, "SpatioTemporalKriging_mean_bc"] = mean_stk
    df_results.loc[allergen, "SpatioTemporalKriging_std_bc"] = std_stk

    # SpatioTemporalKriging(log)
    model_stk = SpatioTemporalKriging(transform='log')
    mean_stk, std_stk = model_stk.cross_validate(df_allergen, meteo_df=meteo_df)
    df_results.loc[allergen, "SpatioTemporalKriging_mean_log"] = mean_stk
    df_results.loc[allergen, "SpatioTemporalKriging_std_log"] = std_stk

    # SpatioTemporalKrigingStandard bez exog(boxcox)
    model_stk = SpatioTemporalKrigingStandard(transform='boxcox')
    mean_stk, std_stk = model_stk.cross_validate(df_allergen)
    df_results.loc[allergen, "SpatioTemporalKrigingStand_mean_bc"] = mean_stk
    df_results.loc[allergen, "SpatioTemporalKrigingStand_std_bc"] = std_stk

    # SpatioTemporalKrigingStandard shared lambda bez exog(log)
    model_stk = SpatioTemporalKrigingStandard(transform='log')
    mean_stk, std_stk = model_stk.cross_validate(df_allergen)
    df_results.loc[allergen, "SpatioTemporalKrigingStand_mean_log"] = mean_stk
    df_results.loc[allergen, "SpatioTemporalKrigingStand_std_log"] = std_stk

    # SpatioTemporalKriging shared lambda sa exog(boxcox)
    model_stk = SpatioTemporalKrigingStandard(transform='boxcox' ,exog_cols=['temperature', 'humidity'])
    mean_stk, std_stk = model_stk.cross_validate(df_allergen, meteo_df)
    df_results.loc[allergen, "SpatioTemporalKrigingStand_mean_exog_bc"] = mean_stk
    df_results.loc[allergen, "SpatioTemporalKrigingStand_std_exog_bc"] = std_stk

    # SpatioTemporalKriging shared lambda sa exog(log)
    model_stk = SpatioTemporalKrigingStandard(transform='log', exog_cols=['temperature', 'humidity'])
    mean_stk, std_stk = model_stk.cross_validate(df_allergen, meteo_df)
    df_results.loc[allergen, "SpatioTemporalKrigingStand_mean_exog_log"] = mean_stk
    df_results.loc[allergen, "SpatioTemporalKrigingStand_std_exog_log"] = std_stk
    
    # NaiveTemporalNearest
    model_naive = NaiveTemporalNearest()
    mean_naive, std_naive = model_naive.cross_validate(df_allergen)
    df_results.loc[allergen, "NaiveTemporalNearest_mean"] = mean_naive
    df_results.loc[allergen, "NaiveTemporalNearest_std"] = std_naive

    # IDWTemporalFallback
    model_idw = IDW()
    mean_idw, std_idw = model_idw.cross_validate(df_allergen)
    df_results.loc[allergen, "IDWTemporalFallback_mean"] = mean_idw
    df_results.loc[allergen, "IDWTemporalFallback_std"] = std_idw

df_results.to_csv('allergen_imputation_model_comparison.csv')

k-ti fold: 5it [01:00, 12.02s/it] 0/25 [00:00<?, ?it/s]
k-ti fold: 5it [00:56, 11.21s/it]
k-ti fold: 5it [00:56, 11.27s/it]
k-ti fold: 5it [00:53, 10.61s/it]
k-ti fold: 5it [00:58, 11.74s/it]
k-ti fold: 5it [00:58, 11.74s/it]
k-ti fold: 5it [00:00,  7.57it/s]
k-ti fold: 5it [00:06,  1.40s/it]
k-ti fold: 5it [00:54, 10.82s/it] 1/25 [05:50<2:20:17, 350.71s/it]
k-ti fold: 5it [00:51, 10.22s/it]
k-ti fold: 5it [00:52, 10.44s/it]
k-ti fold: 5it [00:51, 10.27s/it]
k-ti fold: 5it [00:57, 11.45s/it]
k-ti fold: 5it [00:56, 11.26s/it]
k-ti fold: 5it [00:00,  6.87it/s]
k-ti fold: 5it [00:13,  2.64s/it]
k-ti fold: 5it [00:53, 10.78s/it] 2/25 [11:27<2:11:12, 342.29s/it]
k-ti fold: 5it [00:51, 10.29s/it]
k-ti fold: 5it [00:52, 10.47s/it]
k-ti fold: 5it [00:51, 10.28s/it]
k-ti fold: 5it [00:56, 11.38s/it]
k-ti fold: 5it [00:56, 11.28s/it]
k-ti fold: 5it [00:00,  9.68it/s]
k-ti fold: 5it [00:05,  1.08s/it]
k-ti fold: 5it [00:54, 10.92s/it] 3/25 [16:55<2:03:11, 335.99s/it]
k-ti fold: 5it [00:52, 10.41s

### Rezultat

In [17]:
df_results

,SpatioTemporalKriging_mean_bc,SpatioTemporalKriging_std_bc,SpatioTemporalKriging_mean_log,SpatioTemporalKriging_std_log,SpatioTemporalKrigingStand_mean_bc,SpatioTemporalKrigingStand_std_bc,SpatioTemporalKrigingStand_mean_log,SpatioTemporalKrigingStand_std_log,SpatioTemporalKrigingStand_mean_exog_bc,SpatioTemporalKrigingStand_std_exog_bc,SpatioTemporalKrigingStand_mean_exog_log,SpatioTemporalKrigingStand_std_exog_log,NaiveTemporalNearest_mean,NaiveTemporalNearest_std,IDWTemporalFallback_mean,IDWTemporalFallback_std
AMBROZIJA,0.306999,0.006256,0.393526,0.078355,0.299043,0.009290,0.360244,0.005321,0.302083,0.008152,0.376401,0.005466,0.358434,0.006473,1.150838,0.016236
BOKVICA,0.097453,0.005121,0.110944,0.014627,0.103048,0.005397,0.103484,0.003884,0.100810,0.007754,0.104674,0.003806,0.105626,0.003757,0.135314,0.004305
BREST,0.210686,0.012916,0.251799,0.035960,0.197239,0.008323,0.239741,0.019203,0.196640,0.007969,0.242574,0.013538,0.224203,0.014852,0.429536,0.013249
BREZA,0.384578,0.034244,0.501746,0.144400,0.351694,0.004509,0.541401,0.194748,0.354700,0.004778,0.559849,0.190089,0.447710,0.008049,0.970249,0.014616
DUD,0.436311,0.123553,0.456213,0.143281,0.321793,0.022051,0.379156,0.007849,0.315921,0.008954,0.402575,0.010586,0.359362,0.010501,0.910725,0.023292
GRAB,0.330992,0.056944,0.379987,0.062510,0.282453,0.013028,0.403137,0.128570,0.281378,0.010509,0.426948,0.147263,0.345014,0.017353,0.671406,0.019222
HRAST,0.282085,0.005108,0.436088,0.053442,0.260961,0.009790,0.309517,0.006072,0.266513,0.009647,0.321278,0.006894,0.329773,0.005339,0.629690,0.006406
JASEN,0.245120,0.002359,0.260706,0.012167,0.234462,0.005616,0.274125,0.002571,0.238043,0.005581,0.278712,0.002521,0.274804,0.006922,0.491294,0.005940
JAVOR,0.311777,0.120458,0.309535,0.074959,0.238629,0.017325,0.284072,0.010850,0.241468,0.017112,0.287038,0.011350,0.280025,0.004268,0.538134,0.008371
KONOPLjE,0.149940,0.003849,0.219782,0.041694,0.149996,0.005674,0.186870,0.060123,0.149902,0.006278,0.188301,0.062331,0.167467,0.004404,0.317630,0.010989


In [18]:
df_results[[    "SpatioTemporalKriging_mean_bc",
    "SpatioTemporalKriging_mean_log", 
    "SpatioTemporalKrigingStand_mean_bc", 
    "SpatioTemporalKrigingStand_mean_log",
    "SpatioTemporalKrigingStand_mean_exog_bc", 
    "SpatioTemporalKrigingStand_mean_exog_log", 
    "NaiveTemporalNearest_mean", 
    "IDWTemporalFallback_mean"]].mean()

SpatioTemporalKriging_mean_bc               0.266289
SpatioTemporalKriging_mean_log              0.310074
SpatioTemporalKrigingStand_mean_bc          0.237841
SpatioTemporalKrigingStand_mean_log         0.293447
SpatioTemporalKrigingStand_mean_exog_bc     0.239350
SpatioTemporalKrigingStand_mean_exog_log    0.301823
NaiveTemporalNearest_mean                   0.286967
IDWTemporalFallback_mean                    0.567924
dtype: float64